## Read and clean data as needed

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
for dirname, _, filenames in os.walk('/Resources'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv("Resources/data.csv")

## Descriptives/Statistics Summary

## Logistic Regression

## Nearest Neighbor

## Support Vector Machines

## Kernel SVM

## Naive Bayes

## Decision Tree Algorithm

## Random Forest Classification